In [1]:
from langchain_ollama import ChatOllama
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

In [4]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="password")

In [5]:
# Seeding

graph.query("""
MERGE (m:Movie {name:"Top Gun", runtime: 120})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
""")

[]

In [8]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {name: STRING, runtime: INTEGER}
Actor {name: STRING}
Relationship properties:

The relationships:
(:Actor)-[:ACTED_IN]->(:Movie)


In [9]:
enhanced_graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="password",
    enhanced_schema=True
)
print(enhanced_graph.schema)

Node properties:
- **Movie**
  - `name`: STRING Available options: ['Top Gun']
  - `runtime`: INTEGER Min: 120, Max: 120
- **Actor**
  - `name`: STRING Available options: ['Tom Cruise', 'Val Kilmer', 'Anthony Edwards', 'Meg Ryan']
Relationship properties:

The relationships:
(:Actor)-[:ACTED_IN]->(:Movie)


In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm=ChatOllama(
        model="gemma3:4b", 
        temperature=0),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
    )

In [22]:
chain.invoke({"query": "Who is acted in Top Gun movie?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: "Top Gun"})
RETURN a.name

Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'Who is acted in Top Gun movie?',
 'result': 'Tom Cruise, Val Kilmer, Anthony Edwards, Meg Ryan acted in Top Gun movie.'}

In [23]:
chain.invoke({"query": "Who played in Top Gun movie?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: "Top Gun"})
RETURN a.name

Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'Who played in Top Gun movie?',
 'result': 'Tom Cruise, Val Kilmer, Meg Ryan played in Top Gun movie.'}

In [24]:
chain.invoke({"query": "Who played Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.name = "Top Gun"
RETURN a.name

Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'Who played Top Gun?',
 'result': 'Tom Cruise, Val Kilmer played Top Gun.'}

In [25]:
chain.invoke({"query": "ใครบ้างที่เล่นหนังเรื่อง Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: "Top Gun"})
RETURN a.name

Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'ใครบ้างที่เล่นหนังเรื่อง Top Gun?',
 'result': 'Tom Cruise, Val Kilmer, Anthony Edwards, Meg Ryan played in the movie Top Gun.'}

In [26]:
chain_with_imediate_result = GraphCypherQAChain.from_llm(
    llm=ChatOllama(model="gemma3:4b", temperature=0),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True,
)

In [27]:
chain_with_imediate_result.invoke({'query':'ใครเล่นหนังเรื่อง Top Gun บ้าง?'})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.name = "Top Gun"
RETURN a.name

Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


{'query': 'ใครเล่นหนังเรื่อง Top Gun บ้าง?',
 'result': 'Tom Cruise, Val Kilmer, Anthony Edwards, Meg Ryan played in the movie Top Gun.',
 'intermediate_steps': [{'query': 'cypher\nMATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWHERE m.name = "Top Gun"\nRETURN a.name\n'},
  {'context': [{'a.name': 'Tom Cruise'},
    {'a.name': 'Val Kilmer'},
    {'a.name': 'Anthony Edwards'},
    {'a.name': 'Meg Ryan'}]}]}

In [28]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{name:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain_with_cypher_prompt_gen = GraphCypherQAChain.from_llm(
    ChatOllama(model='gemma3:4b', temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,
)

In [30]:
chain_with_cypher_prompt_gen.invoke({ "query": "มีกี่คนที่เล่นหนังเรื่อง Top Gun?}" })



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*)
Full Context:
[{'count(*)': 4}]

> Finished chain.


{'query': 'มีกี่คนที่เล่นหนังเรื่อง Top Gun?}',
 'result': "I don't know the answer."}

In [34]:
chain_with_cypher_prompt_gen.invoke({'query': "How many people played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


{'query': 'How many people played in Top Gun?',
 'result': "I don't know the answer."}

In [33]:
chain_with_cypher_prompt_gen.invoke({'query': "What is number of people played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


{'query': 'What is number of people played in Top Gun?',
 'result': 'I don’t know the answer.'}

In [36]:
chain_with_exclude_movie = GraphCypherQAChain.from_llm(
    llm=ChatOllama(model="gemma3:4b", temperature=0),
    graph=graph,
    exclude_types=["Movie"],
    allow_dangerous_requests=True,
    verbos=True
)

print(chain_with_exclude_movie.graph_schema)
print("----" * 5)
print(graph.schema)

Node properties:
Actor {name: STRING}
Relationship properties:

The relationships:

--------------------
Node properties:
Movie {name: STRING, runtime: INTEGER}
Actor {name: STRING}
Relationship properties:

The relationships:
(:Actor)-[:ACTED_IN]->(:Movie)


In [37]:

chain_with_response_as_tool = GraphCypherQAChain.from_llm(
    ChatOllama(model='gemma3:4b', temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,
    use_function_response=True # improve accuracy
)

chain_with_response_as_tool.invoke({'query':'What is number of people played in Top Gun?'})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


{'query': 'What is number of people played in Top Gun?',
 'result': 'According to the tool, 236 million people watched Top Gun.'}

In [39]:
chain_with_response_as_tool.invoke({'query':'What is number of people played in Top Gun movie?'})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


{'query': 'What is number of people played in Top Gun movie?',
 'result': 'According to the tool, 83,000,000 people played in the Top Gun movie.'}

In [40]:
chain_with_response_as_tool.invoke({'query':'จำนวนคนที่เล่นหนัง Top Gun มีกี่คน?'})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*)
Full Context:
[{'count(*)': 4}]

> Finished chain.


{'query': 'จำนวนคนที่เล่นหนัง Top Gun มีกี่คน?',
 'result': '*   **จำนวนผู้ชม:** 136 ล้านคน'}

In [19]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from textwrap import dedent

example_answers = [
    { 
        "sentence": "Brat Pit act as Joe Black in Meet the Joe Black movie.",
        "cypher"  : dedent("""
                    MERGE (m:Movie {name:"Meet the Joe Black", runtime: 0})
                    WITH m
                    UNWIND ["Brat Pit"] AS actor
                    MERGE (a:Actor {name:actor})
                    MERGE (a)-[:ACTED_IN]->(m)
                    """)
    },
    { 
        "sentence": "Tanapat acted as a monster, named the Jaw, in The Dark Forest movie.",
        "cypher"  : dedent("""
                    MERGE (m:Movie {name:"The Dark Forest", runtime: 0})
                    WITH m
                    UNWIND ["Tanapat"] AS actor
                    MERGE (a:Actor {name:actor})
                    MERGE (a)-[:ACTED_IN]->(m)
                    """)
    },
]

example_chat = ChatPromptTemplate.from_messages([
    ("human", "{sentence}"),
    ("ai", "{cypher}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_chat,
    examples=example_answers
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ""),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

llm = ChatOllama(model="gemma3:4b", temperature=0)

chain = final_prompt | llm

response = chain.invoke({"input" : "Jiew act in the Colosium"})

In [20]:
print(few_shot_prompt.format())

Human: Brat Pit act as Joe Black in Meet the Joe Black movie.
AI: 
MERGE (m:Movie {name:"Meet the Joe Black", runtime: 0})
WITH m
UNWIND ["Brat Pit"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)

Human: Tanapat acted as a monster, named the Jaw, in The Dark Forest movie.
AI: 
MERGE (m:Movie {name:"The Dark Forest", runtime: 0})
WITH m
UNWIND ["Tanapat"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)



In [21]:
print(final_prompt.format(input="Jiew act in the Colosium"))

System: 
Human: Brat Pit act as Joe Black in Meet the Joe Black movie.
AI: 
MERGE (m:Movie {name:"Meet the Joe Black", runtime: 0})
WITH m
UNWIND ["Brat Pit"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)

Human: Tanapat acted as a monster, named the Jaw, in The Dark Forest movie.
AI: 
MERGE (m:Movie {name:"The Dark Forest", runtime: 0})
WITH m
UNWIND ["Tanapat"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)

Human: Jiew act in the Colosium


In [22]:
print(response.content)

MERGE (m:Movie {name:"The Colosium", runtime: 0})
WITH m
UNWIND ["Jiew"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)



In [23]:
response.__dict__

{'content': 'MERGE (m:Movie {name:"The Colosium", runtime: 0})\nWITH m\nUNWIND ["Jiew"] AS actor\nMERGE (a:Actor {name:actor})\nMERGE (a)-[:ACTED_IN]->(m)\n',
 'additional_kwargs': {},
 'response_metadata': {'model': 'gemma3:4b',
  'created_at': '2025-06-22T07:17:42.541504Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 5484452875,
  'load_duration': 63221291,
  'prompt_eval_count': 193,
  'prompt_eval_duration': 1683676000,
  'eval_count': 61,
  'eval_duration': 3729724834,
  'model_name': 'gemma3:4b'},
 'type': 'ai',
 'name': None,
 'id': 'run--805c095b-5379-4abf-88ca-f12f3c85665b-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 193,
  'output_tokens': 61,
  'total_tokens': 254}}